# **1 ИМПОРТ БИБЛИОТЕК**

In [48]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.models as M
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K

from sklearn.model_selection import train_test_split

# **2 ИМПОРТ ДАННЫХ**

Расшифровка классов (label), к которым принадлежат изображения:

0 T-shirt/top – футболка

1 Trouser – брюки

2 Pullover – свитер

3 Dress – платье

4 Coat – пальто

5 Sandal – сандалия

6 Shirt – рубашка

7 Sneaker – кроссовок

8 Bag – сумка

9 Ankle boot – сапог

In [49]:
df = pd.read_csv('D:/mifi/myvenv/DL/fmnist_train.csv')

## **2.1 ИССЛЕДОВАНИЕ ДАННЫХ**

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17040 entries, 0 to 17039
Columns: 786 entries, label to Id
dtypes: float64(424), int64(362)
memory usage: 102.2 MB


In [51]:
print(f'Количество строк: {df.shape[0]}, количество столбцов: {df.shape[1]}')
df.head()

Количество строк: 17040, количество столбцов: 786


,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784,Id
0,2,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,9,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,6,0,0,0,0,0,0,0,5,0,...,0.0,0.0,30.0,43.0,0.0,0.0,0.0,0.0,0.0,2
3,0,0,0,0,1,2,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3
4,3,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4


In [52]:
# Количество строк с пропусками
rows_with_nulls = df.isnull().any(axis=1).sum()
print(f'Количество строк с пропусками: {rows_with_nulls}') 

Количество строк с пропусками: 1


In [53]:
# Строки с пропусками
df = df.dropna()
print(f'Количество строк: {df.shape[0]}, количество столбцов: {df.shape[1]}')

Количество строк: 17039, количество столбцов: 786


### **Сбалансированность данных по меткам**

In [54]:
df['label'].value_counts()

label
0    1770
7    1761
3    1725
6    1704
1    1700
9    1694
5    1694
2    1677
8    1675
4    1639
Name: count, dtype: int64

Видим, что данные не сильно но разбалансированы, следовательно будем делить датасет на тестовый и валидационный с учетом стратификации

# **3 ПОДГОТОВКА К ОБУЧЕНИЮ**

In [55]:
# Загружаем данные (признаки X и целевую переменную y)
X = df.drop(columns=['label', 'Id'])  # Все колонки, кроме целевой и Id
y = df['label']                # Целевая переменная

# Разделяем данные (70% train, 30% validation)
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [56]:
print(f'Форма обучающего датасета - Количество строк: {x_train.shape[0]}, количество столбцов: {x_train.shape[1]}')
print(f'Форма валидационного датасета - Количество строк: {x_val.shape[0]}, количество столбцов: {x_val.shape[1]}')

Форма обучающего датасета - Количество строк: 11927, количество столбцов: 784
Форма валидационного датасета - Количество строк: 5112, количество столбцов: 784


In [57]:
# Переведем датафрейм, представив признаки в виде матрицы 28*28
x_train = x_train.to_numpy().reshape(-1, 28, 28)
x_val = x_val.to_numpy().reshape(-1, 28, 28)
print(f'Форма обучающего датасета - {x_train.shape}')
print(f'Форма валидационного датасета - {x_val.shape[0]}')

Форма обучающего датасета - (11927, 28, 28)
Форма валидационного датасета - 5112


In [58]:
# one-hot encode для ответов
y_train_oh = keras.utils.to_categorical(y_train, 10)
y_val_oh = keras.utils.to_categorical(y_val, 10)

print(y_train_oh.shape)
print(y_train_oh[:5], y_train[:5])

(11927, 10)
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]] 4908     6
5324     6
15827    8
14280    2
1629     0
Name: label, dtype: int64


# **4 ОБУЧЕНИЕ МОДЕЛИ**

In [59]:
K.clear_session()

model = M.Sequential()
model.add(L.Conv2D(32, kernel_size=5, strides=1, padding='same', input_shape=(28, 28, 1)))
model.add(L.MaxPool2D())
model.add(L.Conv2D(64, kernel_size=5, strides=1, padding='same'))
model.add(L.MaxPool2D())
model.add(L.Conv2D(128, kernel_size=5, strides=1, padding='same'))
model.add(L.MaxPool2D())
model.add(L.Conv2D(256, kernel_size=5, strides=1, padding='same'))
model.add(L.MaxPool2D())
model.add(L.Conv2D(512, kernel_size=5, strides=1, padding='same'))
model.add(L.Flatten())
model.add(L.Dense(10, activation='softmax'))

d:\mifi\myvenv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [60]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 32)     │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 14, 14, 64)     │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 7, 7, 128)      │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 3, 3, 256)      │       819,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 1, 1, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 1, 1, 512)      │     3,277,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,358,922 (16.63 MB)

 Trainable params: 4,358,922 (16.63 MB)

 Non-trainable params: 0 (0.00 B)

In [61]:
model.compile(
    loss='categorical_crossentropy',  # минимизируем кросс-энтропию
    optimizer='adam',
    metrics=['accuracy']  # выводим процент правильных ответов
)

In [62]:
# центрируем и нормируем, так сети будет проще учиться
x_train_float = x_train.astype(np.float32) / 255 - 0.5
x_val_float = x_val.astype(np.float32) / 255 - 0.5

In [63]:
model.fit(
    x_train_float[:, :, :, np.newaxis],
    y_train_oh,
    batch_size=32,
    epochs=5,
    validation_data=(x_val_float[:, :, :, np.newaxis], y_val_oh)
)

Epoch 1/5
373/373 ━━━━━━━━━━━━━━━━━━━━ 25s 63ms/step - accuracy: 0.6499 - loss: 0.9209 - val_accuracy: 0.8275 - val_loss: 0.4850
Epoch 2/5
373/373 ━━━━━━━━━━━━━━━━━━━━ 22s 60ms/step - accuracy: 0.8571 - loss: 0.4004 - val_accuracy: 0.8464 - val_loss: 0.4164
Epoch 3/5
373/373 ━━━━━━━━━━━━━━━━━━━━ 19s 51ms/step - accuracy: 0.8956 - loss: 0.2998 - val_accuracy: 0.8744 - val_loss: 0.3566
Epoch 4/5
373/373 ━━━━━━━━━━━━━━━━━━━━ 18s 49ms/step - accuracy: 0.9074 - loss: 0.2539 - val_accuracy: 0.8595 - val_loss: 0.4208
Epoch 5/5
373/373 ━━━━━━━━━━━━━━━━━━━━ 22s 58ms/step - accuracy: 0.9200 - loss: 0.2197 - val_accuracy: 0.8728 - val_loss: 0.4160


### **Загрузка тестового датасета**

In [64]:
test_df = pd.read_csv('D:/mifi/myvenv/DL/fmnist_test.csv')
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 785 entries, pixel1 to Id
dtypes: int64(785)
memory usage: 59.9 MB


In [65]:
# Приводим к нужному виду
test_np = test_df.drop(columns=['Id']).to_numpy().reshape(-1, 28, 28)
ids = test_df['Id'].to_numpy()

In [66]:
# Делаем предсказание и выбираем одну метку
res_raw = model.predict(test_np)
res = tf.argmax(res_raw, axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step


In [67]:
# Готовим итоговый датафрейм
res_df = pd.DataFrame({
    'Id': ids,
    'label': res.numpy()
})
res_df.head()

,Id,label
0,0,0
1,1,1
2,2,2
3,3,6
4,4,4


# **ЭКСПОРТИРУЕМ РЕЗУЛЬТАТ**

In [68]:
res_df.to_csv('D:/mifi/myvenv/DL/submission.csv', index=False)